In [6]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [53]:
class person:
    def __init__(self,i,gender, orientation, age, sexual_activity):
        # gender 0 : male, 1 : female
        # orientation 0 : straight, 1 : homosexual
        
        
        self.identifier = i
        self.gender = gender
        self.orientation = orientation
        self.age = age
        self.sexual_activity = sexual_activity
        self.disease_status = 0
        self.time_since_infection = 0
        self.number_of_partners = 0
        self.current_partners = {}
    
    def print_state(self):
        print(self.identifier, self.gender, self.orientation)
    
    def add_partner(self, other_person):
        self.current_partners.add(other_person)
        
    def remove_partner(self, other_person):
        self.current_partners.remove(other_person)

class partnership:
    def __init__(self,partnership_type,person1,person2):
        self.type = partnership_type,
        self.persons = [person1,person2]
        
class system:
    def __init__(self):
        self.persons = []
    
    def set_constants(self,
                      f = 0.2,
                      rho = 0.006,
                      sigma1 = 0.0004,
                      sigma2 = 0.1,
                      transmission_male_female_steady = 0.15,
                      transmission_female_male_steady = 0.0625,
                      transmission_male_female_casual = 0.6,
                      transmission_female_male_casual = 0.25,
                      ratio_asymptomatic_men = 0.1,
                      ratio_asymptomatic_women = 0.45,
                      incubation_time_men = 5,
                      incubation_time_women = 10,
                      patient_delay_treatment_men = 5,
                      patient_delay_treatment_women = 8,
                      recovery_rate_asymptomatic_men = 0.0074,
                      recovery_rate_symptomatic_men = 0.04,
                      recovery_rate_asymptomatic_women = 0.0044,
                      recovery_rate_symptomatic_men = 0.03)
    """
    sets constants
    
    Definitions
    f: probability of steady partnership
    rho: probability of forming partnership
    sigma1 : probability of steady partnership separation
    sigma2 : probability of casual partnership separation
    
    
    """
    self.f = f
    self.rho = rho
    self.sigma1 = sigma1
    self.sigma2 = sigma2
    
    self.transmission_male_female_steady = transmission_male_female_steady
    self.transmission_female_male_steady = transmission_female_male_steady
    self.transmission_male_female_casual = transmission_male_female_casual
    self.transmission_female_male_casual = transmission_female_male_casual
    self.ratio_asymptomatic_men = ratio_asymptomatic_men
    self.ratio_asymptomatic_women = ratio_asymptomatic_women
    self.incubation_time_men = incubation_time_men
    self.incubation_time_women = incubation_time_women
    self.patient_delay_treatment_men = patient_delay_treatment_men
    self.patient_delay_treatment_women = patient_delay_treatment_women
    self.recovery_rate_asymptomatic_men = recovery_rate_asymptomatic_men
    self.recovery_rate_symptomatic_men = recovery_rate_symptomatic_men
    self.recovery_rate_asymptomatic_women = recovery_rate_asymptomatic_women
    self.recovery_rate_symptomatic_men = recovery_rate_symptomatic_men
    
    
    def initialize(self,number_of_persons, gender_ratio, queer_ratio):
        for i in range(number_of_persons):
            gender =  0 if np.random.random() > gender_ratio else 1
            orientation = 0 if np.random.random() > queer_ratio else 1
            age = np.random.randint(15,65)
            if age >= 15 and age < 35:
                sexual_activity = 1 if np.random.random() < 0.05 else 0
            else:
                sexual_activity = 0
            
            
            p = person(i,gender, orientation, age, sexual_activity)
            self.persons.append(p)
            
    def print_state(self):
        for p in self.persons:
            p.print_state()
            
    def timestep():
        

In [54]:
s = system()
s.initialize(10,0.5,0.1)


In [55]:
s.print_state()

0 0 0
1 0 0
2 0 0
3 1 1
4 1 0
5 0 0
6 1 0
7 0 0
8 1 0
9 1 1
